In [ ]:
# CLASS DEFINITIONS FOR NEURAL NETWORKS USED IN DEEP GALERKIN METHOD

#%% import needed packages
import tensorflow as tf
class DenseLayer(tf.keras.layers.Layer):
    
    # constructor/initializer function (automatically called when new instance of class is created)
    def __init__(self, output_dim, input_dim, transformation=None):
        '''
        Args:
            input_dim:       dimensionality of input data
            output_dim:      number of outputs for dense layer
            transformation:  activation function used inside the layer; using
                             None is equivalent to the identity map 
        
        Returns: customized Keras (fully connected) layer object 
        '''        
        
        # create an instance of a Layer object (call initialize function of superclass of DenseLayer)
        super(DenseLayer, self).__init__()
        self.output_dim = output_dim
        self.input_dim = input_dim
        self.W = self.add_weight(
            name="W",
            shape=[self.input_dim, self.output_dim],
            initializer=tf.initializers.GlorotUniform(),
            dtype=tf.float32
        )
        
        # Create the bias vector
        self.b = self.add_weight(
            name="b",
            shape=[1, self.output_dim],
            initializer=tf.zeros_initializer(),
            dtype=tf.float32)
        
        if transformation:
            if transformation == "tanh":
                self.transformation = tf.tanh
            elif transformation == "relu":
                self.transformation = tf.nn.relu
            elif transformation == "mish":
                self.transformation = lambda x: x * tf.math.tanh(tf.math.softplus(x))
            elif transformation == "swish":
                self.transformation = lambda x: x * tf.math.sigmoid(x)
            else:
                self.transformation = None
        else:
            self.transformation = None

    def call(self, X):
        '''Compute output of a dense layer for a given input X 
        
        Args:                        
            X: input to layer            
        '''
        
        # compute dense layer output
        S = tf.add(tf.matmul(X, self.W), self.b)
                
        if self.transformation:
            S = self.transformation(S)
        
        return S

def terminal_utility(xy):
    x = xy[:, 0]
    y = xy[:, 1]
    result = x - tf.exp(y)
    return tf.expand_dims(result, axis=-1)
  
class DGMNet(tf.keras.Model):
    def __init__(self, layer_width, n_layers, input_dim, final_trans=None, feedforward=False, output_dim=1, control_output=False):
        super(DGMNet, self).__init__()

        self.n_layers = n_layers
        self.feedforward = feedforward
        self.output_dim = output_dim

        # Define kernel layers
        self.kernel_layers = []
        for _ in range(n_layers):
            self.kernel_layers.append(DenseLayer(layer_width, layer_width, transformation='swish'))

        # Final output layer
        self.output_weight = tf.keras.layers.Dense(output_dim, activation=final_trans)
        self.control_output = control_output
        self.initial_layer = DenseLayer(layer_width, input_dim+1, transformation='swish')



    def call(self, t, x):
        # Concatenate time and space
        X = tf.concat([t, x], axis=1)
        S = self.initial_layer.call(X)
        for i in range(self.n_layers):
            S = self.kernel_layers[i](S)+S

        # Final output
        result = self.output_weight(S)
        if self.control_output == False:
            result = terminal_utility(x)+result*(1-t)
        return result

In [ ]:
#%% import needed packages
import numpy as np
import matplotlib.pyplot as plt
import time

#%% Parameters 
d = 2 # dimension
sigma = 1  # asset volatility
T = 1        # terminal time (investment horizon)

# Solution parameters (domain on which to solve PDE)
t_low = 0.0    # time lower bound
X_low = 0.0    # wealth lower bound
X_high = 1.0           # wealth upper bound

# neural network parameters
num_layers = 3
nodes_per_layer = 32
starting_learning_rate = 0.001

# Training parameters
sampling_stages  = 50000  # number of times to resample new time-space domain points
steps_per_sample = 10    # number of SGD steps to take before re-sampling

# Sampling parameters
nSim_interior = 2000
nSim_interior_val = 2000
nSim_terminal = 1

# multipliers for oversampling i.e. draw X from [X_low - X_oversample, X_high + X_oversample]
X_oversample = 0.5
t_oversample = 0.0

# Plot options
n_plot = 41  # Points on plot grid for each dimension


#%% Analytical Solution
def h(t, xy):
    x = xy[:, 0][:, np.newaxis]  
    y = xy[:, 1][:, np.newaxis]  
    term1 = x - np.exp(y)
    term2 = 0.25 * np.exp(-y) * x * x
    term3 = np.exp(sigma**2 * (T - t)) - 1
    return term1 + term2 * term3

def terminal_utility(xy):
    x = xy[:, 0]
    y = xy[:, 1]
    result = x - tf.exp(y)
    return tf.expand_dims(result, axis=-1)  

def sampler(nSim_interior, nSim_terminal):
    ''' Sample time-space points from the function's domain; points are sampled
        uniformly on the interior of the domain, at the initial/terminal time points
        and along the spatial boundary at different time points. 
    
    Args:
        nSim_interior: number of space points in the interior of the function's domain to sample 
        nSim_terminal: number of space points at terminal time to sample (terminal condition)
    '''
    # Sampler #1: domain interior    
    t_interior = np.random.uniform(low=t_low - t_oversample*(T-t_low), high=T, size=[nSim_interior, 1]).astype(np.float32)
    X_interior = np.random.uniform(low=X_low - X_oversample*(X_high-X_low), high=X_high + X_oversample*(X_high-X_low), size=[nSim_interior, d]).astype(np.float32)
    

    # Sampler #3: initial/terminal condition
    t_terminal = T * np.ones((nSim_terminal, 1)).astype(np.float32)
    X_terminal = np.random.uniform(low=X_low - X_oversample*(X_high-X_low), high=X_high + X_oversample*(X_high-X_low), size = [nSim_terminal, d]).astype(np.float32)


    return t_interior, X_interior, t_terminal, X_terminal



#%% Loss function for Merton Problem PDE

def loss(model,control, t_interior, X_interior, t_terminal, X_terminal):
    ''' Compute total loss for training.'''

    with tf.GradientTape(persistent=True, watch_accessed_variables=False) as gt:
        gt.watch(t_interior)
        gt.watch(X_interior)
        V = model(t_interior, X_interior)  # V shape: (?, 1)
        V_1 = gt.gradient(V, X_interior)   # First-order gradients, shape: (?, 2)
        
    V_t = gt.gradient(V, t_interior)  # Time derivative, shape: (?, 1)
    Z = control(t_interior, X_interior)  # Control function, shape: (?, 1)
    V_1_x = tf.expand_dims(V_1[:, 0], axis=-1)  # Shape: [4, 1]
    V_1_y = tf.expand_dims(V_1[:, 1], axis=-1)
    V_2 = gt.batch_jacobian(V_1, X_interior)  # shape (?, 2, 2)
    V_xx = tf.expand_dims(V_2[:, 0, 0], axis=-1)  # d²V/dx²
    V_yy = tf.expand_dims(V_2[:, 1, 1], axis=-1)  # d²V/dy²
    V_xy = tf.expand_dims(V_2[:, 0, 1], axis=-1)  # d²V/dxdy

    # Extract x and y
    x_x = tf.expand_dims(X_interior[:, 0], axis=-1)  #  (?, 1)
    x_y = tf.expand_dims(X_interior[:, 1], axis=-1)  #  (?, 1)

    # Compute terms
    term1 = sigma * sigma * x_x * x_x * Z * V_1_x  # shape (?, 1)
    term2 = 0.5 * sigma * sigma * x_x * x_x * Z * Z * V_1_y  # shape (?, 1)
    term3 = 0.5 * sigma * sigma * x_x * x_x * (V_xx + Z * Z * V_yy)  # shape (?, 1)
    term4 = sigma * sigma * x_x * x_x * Z * V_xy  # shape (?, 1)

    # Compute the PDE residual
    diff_V = V_t + term1 + term2 + term3 + term4  # shape (?, 1)

    L1 = tf.reduce_mean(tf.square(diff_V))  # Loss term for PDE


    target_terminal = terminal_utility(X_terminal)
    fitted_terminal = model(t_terminal, X_terminal)
    diff_terminal = fitted_terminal - target_terminal
    L3 = tf.reduce_mean(tf.square(fitted_terminal - target_terminal) )  # Loss term for terminal condition
    
    del gt
    return L1, L3, diff_V, diff_terminal

def loss_control(model, control, t_interior, X_interior, t_terminal, X_terminal):
    ''' Compute total loss for training.'''
    with tf.GradientTape(persistent=True, watch_accessed_variables=False) as gt:
        gt.watch(t_interior)
        gt.watch(X_interior)
        V = model(t_interior, X_interior)  # V shape: (?, 1)
        V_1 = gt.gradient(V, X_interior)   # First-order gradients, shape: (?, 2)
        
    V_t = gt.gradient(V, t_interior)  # Time derivative, shape: (?, 1)
    Z = control(t_interior, X_interior)  # Control function, shape: (?, 1)
    V_1_x = tf.expand_dims(V_1[:, 0], axis=-1)  # Shape: [4, 1]
    V_1_y = tf.expand_dims(V_1[:, 1], axis=-1)
    # Compute second-order derivatives
    V_2 = gt.batch_jacobian(V_1, X_interior)  # shape (?, 2, 2)

    V_xx = tf.expand_dims(V_2[:, 0, 0], axis=-1)  # d²V/dx²
    V_yy = tf.expand_dims(V_2[:, 1, 1], axis=-1)  # d²V/dy²
    V_xy = tf.expand_dims(V_2[:, 0, 1], axis=-1)  # d²V/dxdy

    # Extract x and y
    x_x = tf.expand_dims(X_interior[:, 0], axis=-1)  # (?, 1)
    x_y = tf.expand_dims(X_interior[:, 1], axis=-1)  # (?, 1)

    # Compute terms
    term1 = sigma * sigma * x_x * x_x * Z * V_1_x  # shape (?, 1)
    term2 = 0.5 * sigma * sigma * x_x * x_x * Z * Z * V_1_y # shape (?, 1)
    term3 = 0.5 * sigma * sigma * x_x * x_x * (V_xx + Z * Z * V_yy)  # shape (?, 1)
    term4 = sigma * sigma * x_x * x_x * Z * V_xy  # shape (?, 1)

    # Compute the PDE residual
    f = term1 + term2 + term3 + term4  # shape (?, 1)

    L2 = tf.reduce_mean(f)  # Loss term for control
    dfdz = sigma * sigma * x_x * x_x * V_1_x + \
           sigma * sigma * x_x * x_x * Z * V_1_y + \
            sigma * sigma * x_x * x_x * Z * V_yy + \
            sigma * sigma * x_x * x_x * V_xy
    dfdz = tf.reduce_mean(dfdz) # derivative of f(L2) on control
    del gt
    return -L2, dfdz

#%% Set up network

# initialize DGM model
model = DGMNet(nodes_per_layer, num_layers, input_dim=d, output_dim=1)
control = DGMNet(nodes_per_layer, num_layers, input_dim=d,output_dim=1, control_output=True)


lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=0.001, decay_steps=100000, end_learning_rate=0.00001, power=0.8
)

# Optimizers
optimizer_value = tf.keras.optimizers.Adam(lr_schedule)
optimizer_control = tf.keras.optimizers.Adam(lr_schedule)


#%% Train network

# Define the train_step function
@tf.function
def train_step(model, control, t_interior, X_interior, t_terminal, X_terminal, optimizer_value, optimizer_control):
    ''' Perform of single training step.'''
    # Compute loss for the value function (L1 + L3)
    t_interior = tf.convert_to_tensor(t_interior, dtype=tf.float32)
    X_interior = tf.convert_to_tensor(X_interior, dtype=tf.float32)
    t_terminal = tf.convert_to_tensor(t_terminal, dtype=tf.float32)
    X_terminal = tf.convert_to_tensor(X_terminal, dtype=tf.float32)

    with tf.GradientTape() as tape:
        L1, L3, diff_V, diff_terminal = loss(model, control, t_interior, X_interior, t_terminal, X_terminal)
        total_loss = L1+L3
    grads = tape.gradient(total_loss, model.trainable_variables)

    optimizer_value.apply_gradients(zip(grads, model.trainable_variables))  # Update the value model
    
    # Compute loss for the control function (L2)
    with tf.GradientTape() as tape1:
        L2,_ = loss_control(model, control, t_interior, X_interior, t_terminal, X_terminal)
    grads_control = tape1.gradient(L2, control.trainable_variables)
    optimizer_control.apply_gradients(zip(grads_control, control.trainable_variables))  # Update the control model
    
    return L1, L3, diff_V, diff_terminal, L2, total_loss


@tf.function
def eval_model(model, control, t_interior_val, X_interior_val, t_terminal_val, X_terminal_val):
    L1_val, L3_val, diff_V_val, diff_terminal_val = loss(model, control, t_interior_val, X_interior_val, t_terminal_val, X_terminal_val)
    L2_val, dL2dZ_val = loss_control(model, control, t_interior_val, X_interior_val, t_terminal_val, X_terminal_val)
    return L1_val, L3_val, diff_V_val, diff_terminal_val, L2_val, dL2dZ_val

# Initialize lists for storing loss values
loss_list = []
L2_list = []
L1_list = []
L3_list = []
value_list = []
control_list = []
diffV_list = []
diffZ_list = []

maxdiff_V = 100
maxdiff_terminal = 100

# dataset = dataset.prefetch(tf.data.AUTOTUNE)
t_eval = tf.constant([[0.0]], dtype=tf.float32)  # shape (1, 1)
X_eval = tf.constant([[1.0, 0.0]], dtype=tf.float32)  # shape (1, 3)

time_record = []
start_time = time.time()
# Main training loop
for i in range(sampling_stages):
    t_interior, X_interior, t_terminal, X_terminal = sampler(nSim_interior, nSim_terminal)
    for _ in range(steps_per_sample):
        L1, L3, diff_V, diff_terminal, L2, total_loss = train_step(
                model, control, t_interior, X_interior,
                t_terminal, X_terminal,
                optimizer_value, optimizer_control
            )
    loss_list.append(total_loss)
    L2_list.append(L2)
    L1_list.append(L1)
    L3_list.append(L3)

    # val
    t_interior_val, X_interior_val, t_terminal_val, X_terminal_val = sampler(nSim_interior_val, nSim_terminal)
    t_interior_val = tf.convert_to_tensor(t_interior_val, dtype=tf.float32)
    X_interior_val = tf.convert_to_tensor(X_interior_val, dtype=tf.float32)
    t_terminal_val = tf.convert_to_tensor(t_terminal_val, dtype=tf.float32)
    X_terminal_val = tf.convert_to_tensor(X_terminal_val, dtype=tf.float32)

    L1_val, L3_val, diff_V_val, diff_terminal_val, L2_val, dL2dZ_val = eval_model(model, control, t_interior_val, X_interior_val, t_terminal_val, X_terminal_val)

    maxdiff_V = np.max(np.abs(diff_V_val.numpy()))
    maxdiff_terminal = np.max(np.abs(diff_terminal_val.numpy()))
    maxdL2dZ = np.max(np.abs(dL2dZ_val.numpy()))
    diffV_list.append(maxdiff_V)
    diffZ_list.append(maxdL2dZ)

    print(f"Stage {i}:{maxdiff_V},{maxdiff_terminal},{maxdL2dZ},L1 = {L1.numpy()}, L3 = {L3.numpy()}, L2 = {L2.numpy()}")

    # Evaluate at (t=0, x=1, y=0)
    V_pred = model(t_eval, X_eval)
    value_list.append(V_pred.numpy()[0, 0])
    print("V(0, 1, 0) =", V_pred.numpy()[0, 0])
    control_pred = control(t_eval, X_eval)
    control_list.append(control_pred.numpy()[0, 0])
    print("z(0, 1, 0) =", control_pred.numpy()[0, 0])
    end_time = time.time()
    time_record.append(end_time - start_time)
    if maxdiff_V < 1e-3 and maxdL2dZ < 1e-3:
        break


In [ ]:
import pandas as pd
def to_float_list(tensor_list):
    return [float(t.numpy()) if isinstance(t, tf.Tensor) else t for t in tensor_list]

df = pd.DataFrame({
    'diffV': to_float_list(diffV_list),
    'diffZ': to_float_list(diffZ_list),
    'loss': to_float_list(loss_list),
    'L1': to_float_list(L1_list),
    'L3': to_float_list(L3_list),
    'L2': to_float_list(L2_list),
    'V_pred': value_list,
    'control_pred': control_list,
})

from datetime import datetime
now = datetime.now().strftime("%Y%m%d_%H%M%S")
df.to_csv(f'result_{now}.csv', index=False)

In [ ]:
x1 = np.linspace(X_low, X_high, n_plot**2)
x2 = np.linspace(X_low, X_high, n_plot**2)
X1, X2 = np.meshgrid(x1, x2)

# Create mesh for t=1 and the x coordinates (flattened)
t_mesh = np.ones((n_plot**2* n_plot**2, 1), dtype=np.float32)  # t=1
x_mesh = np.stack([X1.flatten(), X2.flatten()], axis=1).astype(np.float32)

# Evaluate the model output (Numerical solution)
h_numerical = model(t_mesh, x_mesh)
# Evaluate the analytical solution
h_analytical = h(1, x_mesh)

# Convert to NumPy arrays for further handling
h_numerical = h_numerical.numpy().reshape(n_plot**2, n_plot**2)
h_analytical = h_analytical.reshape(n_plot**2, n_plot**2)
print(h_numerical,h_analytical)

# Compute the absolute error between numerical and analytical solutions
error = np.abs(h_numerical - h_analytical)

# Plot the error as a heatmap
plt.figure(figsize=(10, 8))
plt.pcolormesh(X1, X2, error, cmap='rainbow', shading='auto')
plt.colorbar(label='Error: h_numerical - h_analytical')
plt.xlabel('x1', fontsize=14)
plt.ylabel('x2', fontsize=14)
plt.title('Absolute Error of H(1,x)', fontsize=16)
plt.show()

In [ ]:
x1 = np.linspace(X_low, X_high, n_plot**2)
x2 = np.linspace(X_low, X_high, n_plot**2)
X1, X2 = np.meshgrid(x1, x2)

# Create mesh for t=1 and the x coordinates (flattened)
t_mesh = np.zeros((n_plot**2*n_plot**2, 1), dtype=np.float32)  # t=1
x_mesh = np.stack([X1.flatten(), X2.flatten()], axis=1).astype(np.float32)

# Evaluate the model output (Numerical solution)
h_numerical = model(t_mesh, x_mesh)


h_analytical = h(0, x_mesh)

# Convert to NumPy arrays for further handling
h_numerical = h_numerical.numpy().reshape(n_plot**2, n_plot**2)
h_analytical = h_analytical.reshape(n_plot**2, n_plot**2)
print(h_numerical,h_analytical)

# Compute the absolute error between numerical and analytical solutions
error = np.abs(h_numerical - h_analytical)

# Plot the error as a heatmap
plt.figure(figsize=(10, 8))
plt.pcolormesh(X1, X2, error, cmap='rainbow', shading='auto')
plt.colorbar(label='Error: h_numerical - h_analytical')
plt.xlabel('x1', fontsize=14)
plt.ylabel('x2', fontsize=14)
plt.title('Absolute Error of H(0,x)', fontsize=16)
plt.show()

In [ ]:
def c(t, xy):
    x = xy[:, 0][:, np.newaxis]  
    y = xy[:, 1][:, np.newaxis]
    return 1.0 / (2.0 * np.exp(y))
x1 = np.linspace(X_low, X_high, n_plot**2)
x2 = np.linspace(X_low, X_high, n_plot**2)
X1, X2 = np.meshgrid(x1, x2)

# Create mesh for t=1 and the x coordinates (flattened)
t_mesh = np.zeros((n_plot**2*n_plot**2, 1), dtype=np.float32)  # t=1
x_mesh = np.stack([X1.flatten(), X2.flatten()], axis=1).astype(np.float32)

# Evaluate the model output (Numerical solution)
z_numerical = control(t_mesh, x_mesh)
z_analytical = c(0, x_mesh)

# Convert to NumPy arrays for further handling
z_numerical = z_numerical.numpy().reshape(n_plot**2, n_plot**2)
z_analytical = z_analytical.reshape(n_plot**2, n_plot**2)
print(z_numerical,z_analytical)

# Compute the absolute error between numerical and analytical solutions
error = np.abs(z_numerical - z_analytical)

# Plot the error as a heatmap
plt.figure(figsize=(10, 8))
plt.pcolormesh(X1, X2, error, cmap='rainbow', shading='auto')
plt.colorbar(label='Error: z_numerical - z_analytical')
plt.xlabel('x1', fontsize=14)
plt.ylabel('x2', fontsize=14)
plt.title('Absolute Error of Z(0,x)', fontsize=16)
plt.show()

In [ ]:
from datetime import datetime

now = datetime.now().strftime("%Y%m%d_%H%M%S")

# save model weights
model.save_weights(f"model_{now}.weights.h5")
control.save_weights(f"modelcontrol_{now}.weights.h5")